# 0. Python packages

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

import testfunctions

# 1. Input

In [ ]:
# Model space dimension.
dim=10
# Initial inverse mass matrix.
Minv=np.identity(dim)
# Number of iterations for the test.
it=10
# Maximum number of LF-BFGS vectors.
k=10

# 2. Functions for various versions of BFGS updating

## 2.1. Regular BFGS updating

In [ ]:
class bfgs:
    
    def __init__(self,dim,Minv,m,g):
        """
        Initialise the BFGS iteration.
        
        :param dim: number of model-space dimensions
        :param Minv: initial inverse mass matrix
        :param m: current model vector
        :param g: current gradient
        
        The matrix Minv plays the role of the inverse mass matrix, which ideally is the inverse Hessian, i.e., the covariance matrix.
        """
        
        self.dim=dim
        self.m=m
        self.g=g
        
        # Initial mass matrix.
        self.Minv=Minv
        
        # Initial factorisation.
        LT=np.linalg.cholesky(self.Minv).transpose()
        self.LTinv=np.linalg.inv(LT)
        
        
    def update(self,m,g):
        """
        Update BFGS matrix and perform Cholesky decomposition.
        
        :param m: current model vector
        :param g: current gradient
        """
        
        # Compute differences and update vectors.
        s=m-self.m
        y=g-self.g

        if np.dot(s,y)>0:
        
            self.m=m
            self.g=g
        
            # Compute update of BFGS matrix.
            rho=1.0/np.dot(s,y)
            I=np.identity(self.dim)
            sy=rho*np.tensordot(s,y,axes=0)
            ss=rho*np.tensordot(s,s,axes=0)
            self.Minv=np.matmul(np.matmul((I-sy),self.Minv),(I-sy.transpose()))+ss
        
            # Compute Cholesky decomposition.
            LT=np.linalg.cholesky(self.Minv).transpose()
            self.LTinv=np.linalg.inv(LT)
            
        else: 
            rhoinv=np.dot(s,y)
            print('BFGS check failed (1/rho=%f)' % rhoinv)

## 2.2. Factorised BFGS updating 

In [ ]:
class fbfgs:
    
    def __init__(self,dim,Minv,m,g,d_max):
        """
        Initialise the BFGS iteration.
        
        :param dim: number of model-space dimensions
        :param Minv: initial inverse mass matrix (must be diagonal)
        :param m: current model vector
        :param g: current gradient
        :param d_max: maximum change of the determinant of the Hessian factor S in of F-BFGS update
        """
        
        self.dim=dim   # Model space dimension.
        self.Hinv=Minv   # Initial (current) estimate of the inverse Hessian H^{-1}.
        
        # Initial (current) matrix factor S and estimate of the Hessian
        
        self.S=np.identity(dim)   
        self.H=np.identity(dim)   
        
        for i in range(dim):
            self.H[i,i]=1.0/Minv[i,i]
            self.S[i,i]=1.0/np.sqrt(Minv[i,i])
        
        self.logdet=0.0   # Initial (current) logarithm of the determinant of the Hessian H.
        self.m=m   # Initial (current) model.
        self.g=g   # Initial (current) gradient.
        self.I=np.identity(dim)   # Identity matrix. 
        
        self.sigma=d_max   # Minimum factorial change of determinant per update.
        
    def update(self,m,g):
        """
        Update BFGS matrix and its factorised form.
        
        :param m: current model vector
        :param g: current gradient
        """
        
        # Compute differences and update vectors.
        s=m-self.m
        y=g-self.g
        
        # Do nothing unless rho is positive.
        if np.dot(s,y)>0:
            
            # Set new to current vectors.
            self.m=m
            self.g=g
            
            # Precompute inverse Hessian-vector product.
            Hinv_y=np.dot(self.Hinv,y)
            
            # Auxiliary scalars.
            rho=1.0/np.dot(s,y)
            gamma2=rho**2 * np.dot(y,Hinv_y) + rho
            beta=gamma2 * np.dot(s,np.dot(self.H,s))
            theta=-np.sqrt(rho/(beta*gamma2))
        
            # Auxiliary vectors a, b, and u, v.
            a=np.sqrt(gamma2)*s
            b=(rho/np.sqrt(gamma2))*Hinv_y
            u=a
            v=-np.dot(self.H,b+theta*a)
            
            # Auxiliary scalar for regularisation.
            #sigma_threshold=(1.0/(1.0+np.dot(u,v)))**2
            
            # Scaling of v for regularisation.
            #if sigma_threshold<self.sigma:
            #    r=(1.0-self.sigma)/(self.sigma*np.dot(u,v))
            #    v=r*v
        
            # Update inverse Hessian estimate Hinv. 
            alpha=1.0/(1.0+np.dot(u,v))
            Hinv_v=np.dot(self.Hinv,v)
            self.Hinv=self.Hinv+np.tensordot(Hinv_v,u,axes=0)+np.tensordot(u,Hinv_v,axes=0)+np.tensordot(u,u,axes=0)*np.dot(v,Hinv_v)
        
            # Update Hessian estimate H.
            H_u=np.dot(self.H,u)
            self.H=self.H-alpha*np.tensordot(H_u,v,axes=0)-alpha*np.tensordot(v,H_u,axes=0)+(alpha**2)*np.tensordot(v,v,axes=0)*np.dot(u,H_u)
        
            # Update factor S.
            ST_u=np.dot(self.S.transpose(),u)
            self.S=self.S-alpha*np.tensordot(v,ST_u,axes=0)
            
            # Update determinant of S.
            self.logdet=self.logdet + np.log(alpha**2)
            
        else: 
            rhoinv=np.dot(s,y)
            print('F-BFGS check failed (1/rho=%f)' % rhoinv)

## 2.3. Limited-memory factorised BFGS

In [ ]:
class lfbfgs:
    
    # Initialisation. ==================================================================
    
    def __init__(self,dim,Minv,k,m,g,d_max):
        """
        Initialise the LF-BFGS iteration.
        
        :param dim: number of model-space dimensions
        :param Minv: diagonal elements of the initial inverse mass matrix
        :param k: maximum number of vectors to be stored
        :param m: current model vector
        :param g: current gradient
        :param d_max: maximum change of the determinant of the Hessian factor S in of F-BFGS update
        """
        
        self.dim=dim # Model space dimension.
        self.k=k # Maximum number of vectors.
        self.i=0 # Currently stored vectors.
        
        self.m=m # Initial (current) model.
        self.g=g # Initial (current) gradient.
        
        self.s=np.zeros((dim,k)) # s vectors.
        self.y=np.zeros((dim,k)) # y vectors.
        self.u=np.zeros((dim,k)) # u vectors.
        self.v=np.zeros((dim,k)) # v vectors.
        self.vTu=np.zeros(k) # Precomputed 1+vT*u.
        
        self.S0=np.ones(dim) # Diagonal components of the initial S factor.
        for i in range(dim):
            self.S0[i]=1.0/np.sqrt(Minv[i])
        
        self.sigma=d_max   # Minimum factorial change of determinant per update.
        
    # Compute and store new vectors u and v. ===========================================
        
    def put_uv(self,m,g):
        """
        Put in a new pair of vectors u and v.
        
        :param m: current model vector
        :param g: current gradient
        """
        
        #===============================================================================
        # Compute new s, y, u, v vectors.
        #===============================================================================
        
        # Compute differences to get the current s and y.
        s=m-self.m
        y=g-self.g
        
        sy=np.dot(s,y)
        
        # Do nothing unless rho is positive.
        if sy>0:
            
            # Set new to current vectors.
            self.m=m
            self.g=g
            
            # Precompute inverse Hessian-vector product.
            Hinv_y=self.Hinv(y)
        
            # Auxiliary scalars.
            rho=1.0/sy
            gamma2=rho**2 * np.dot(y,Hinv_y) + rho
            beta=gamma2 * np.dot(s,self.H(s))
            theta=-np.sqrt(rho/(beta*gamma2)) # Here, one may also choose the positive square root. Empirically, the negative ones works much better because it leads to a diagonally dominant S, which is better for preconditioning.
        
            # Auxiliary vectors a and b.
            a=np.sqrt(gamma2)*s
            b=(rho/np.sqrt(gamma2))*Hinv_y
        
            # Compute the current u and v.
            u=a
            v=-self.H(b+theta*a)
            
            # Auxiliary scalar for regularisation.
            #sigma_threshold=(1.0/(1.0+np.dot(u,v)))**2
            
            # Scaling of v for regularisation.
            #if sigma_threshold<self.sigma:
            #    r=(1.0-self.sigma)/(self.sigma*np.dot(u,v))
            #    v=r*v
            
            #===============================================================================
            # Update s, y, u, v in the memory.
            #===============================================================================
        
            # When less then k vector pairs are stored, we increase the index of stored pairs and add the new pair.
            if self.i<self.k:
                self.i+=1
            # Otherwise we kick out the pair with the lowest index by rolling to the left and over-writing the last pair.
            else:
                self.s=np.roll(self.s, -1, axis=1)
                self.y=np.roll(self.y, -1, axis=1)
                self.u=np.roll(self.u, -1, axis=1)
                self.v=np.roll(self.v, -1, axis=1)
                self.vTu=np.roll(self.vTu,-1)
        
            self.s[:,self.i-1]=s
            self.y[:,self.i-1]=y
            self.u[:,self.i-1]=u
            self.v[:,self.i-1]=v
            self.vTu[self.i-1]=1.0+np.dot(v,u)
            
        else: 
            print('LF-BFGS check failed (1/rho=%f)' % sy)
            
    # Update diagonal elements of the initial factor S0. ===============================

    def update_S0(self):
        """
        Update the diagonal elements of the initial factor S0 using the diagonal elements of the current approximation.
        """
    
        #p=np.ones(self.dim)
        #self.S0=np.abs(self.S(p))
        #print(np.min(self.S0))
    
        S0=np.zeros(self.dim)
        for i in range(self.dim):
            p=np.zeros(self.dim)
            p[i]=1.0
            S0[i]=self.S(p)[i]
        
        #if np.min(S0) < 0.0: S0=np.ones(self.dim)
        print("minS0=%f" % np.min(S0))
        self.S0=S0
        
    # Reset diagonal elements of the initial factor S0. ===============================
    
    def reset_S0(self):
        """
        Reset the diagonal elements of S0 to 1.
        """
        self.S0=np.ones(dim)
            

    # Implement matrix-vector products. ===============================================
    
    def S(self,h):
        """
        S*h, product of factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i):
            h=h-self.v[:,i]*np.dot(self.u[:,i],h)/self.vTu[i]
        return h*self.S0
    
    def Sn(self,h,n):
        """
        S*h, product of factor S with some vector h using a smaller number n of vectors
        :param h: vector of dimension self.dim
        :param n: number of vectors to take into account
        """
        
        for i in range(self.i):
            h=h-self.v[:,i]*np.dot(self.u[:,i],h)/self.vTu[i]
        return h*self.S0
    
    def ST(self,h):
        """
        S^T*h, product of transposed factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i-1,-1,-1):
            h=h-self.u[:,i]*np.dot(self.v[:,i],h)/self.vTu[i]
        return h*self.S0
        
    def Sinv(self,h):
        """
        S^-1*h, product of inverse factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i-1,-1,-1):
            h=h+self.v[:,i]*np.dot(self.u[:,i],h)
        return h/self.S0
    
    def SinvT(self,h):
        """
        S^-T*h, product of inverse transposed factor S with some vector h
        :param h: vector of dimension self.dim
        """
        
        for i in range(self.i):
            h=h+self.u[:,i]*np.dot(self.v[:,i],h)
        return h/self.S0
    
    def H(self,h):
        """
        H*h, product of Hessian approximation H with some vector h
        :param h: vector of dimension self.dim
        """
        
        h=self.ST(h)
        return self.S(h)
    
    def Hinv(self,h):
        """
        Hinv*h, product of inverse Hessian approximation H with some vector h
        :param h: vector of dimension self.dim
        """
        
        h=self.Sinv(h)
        return self.SinvT(h)
    
    # Implement log determinant of S. ==================================================
    def logdet(self):
        """
        Current log of the determinant of S
        """
        
        logdet=0.0
        for i in range(self.i):
            alpha=1.0/(1.0+np.dot(self.u[:,i],self.v[:,i]))
            logdet+=np.log(alpha**2)
            
        return logdet

# 3. Run test by generating sequence of $m$ and $g$

In [ ]:
# Initial m and g.
m=np.random.randn(dim)
g=np.random.randn(dim)

# Initialise BFGS updating schemes.
M_bfgs=bfgs(dim,Minv,m,g)
M_fbfgs=fbfgs(dim,Minv,m,g,1.0)
M_lfbfgs=lfbfgs(dim,Minv.diagonal(),k,m,g,1.0)

In [ ]:
for i in range(it):
    # New m and g.
    m_new=np.random.randn(dim)
    g_new=np.random.randn(dim)
    # Check updating condition.
    rhoinv=np.dot(m-m_new,g-g_new)
    print('1/rho=%f' % rhoinv)
    # Update
    M_bfgs.update(m_new,g_new)
    M_fbfgs.update(m_new,g_new)
    M_lfbfgs.put_uv(m_new,g_new)

# 4. Analyse output

In [ ]:
# Random test vector.
p=np.random.randn(dim)

# Compute products with current mass matrix M.
prod_bfgs=np.dot(M_bfgs.LTinv,np.dot(M_bfgs.LTinv.transpose(),p))
prod_fbfgs=np.dot(M_fbfgs.H,p)
prod_lfbfgs=M_lfbfgs.H(p)

offset=np.max(np.abs(prod_bfgs))/50.0

plt.plot(prod_bfgs,'k')
plt.plot(prod_fbfgs+1.0*offset,'b')
plt.plot(prod_lfbfgs+2.0*offset,'r')
plt.show()

print(prod_bfgs)
print(prod_fbfgs)
print(prod_lfbfgs)

# Compute products with inverse of current mass matrix M^-1.
prod_bfgs=np.dot(M_bfgs.Minv,p)
prod_fbfgs=np.dot(M_fbfgs.Hinv,p)
prod_lfbfgs=M_lfbfgs.Hinv(p)

offset=np.max(np.abs(prod_bfgs))/50.0

plt.plot(prod_bfgs,'k')
plt.plot(prod_fbfgs+1.0*offset,'b')
plt.plot(prod_lfbfgs+1.0*offset,'r')
plt.show()

print(prod_bfgs)
print(prod_fbfgs)
print(prod_lfbfgs)